# Зависимости

In [ ]:
from functions import (load_portion_functions, append_functions_infile,
                       filter_functions_name)
from settings import FUNCTIONS_PATH, FILTERED_FUNCTIONS_PATH

# Выполнение анализа имен функций

`uniq_names` - словарь уникальных значений функций со стчетчиком

In [ ]:
uniq_names = {}

### Подсчет уникальных имен функций

Из-за большого размера файлов оперативная память не справиться с обработкой, поэтому производится порциональный анализ функций.

* batch - размер порции (1000 для ОЗУ объемом 4 Гб)

In [ ]:
batch = 1000
pos = 0
functions, pos = load_portion_functions(FUNCTIONS_PATH, pos, 1)

print('Progress:')
while True:
    functions, pos = load_portion_functions(FUNCTIONS_PATH, pos, batch)
    if not functions:
        break

    # Упростить названия функций
    for i,f in enumerate(functions):
        functions[i][0]['value'] = functions[i][0]['value'].lower().replace('__', '')
        
    # Добавить имена в словарь подсчета
    for func in functions:
        uniq_names[func[0]['value']] = uniq_names.get(func[0]['value'], 0) + 1
    
    print('.', end='')

In [ ]:
print('Уникальных функций:', len(uniq_names))

### Выбор самых частотных имен функций

* thresholder - задает порог вхождения функции в выборку

In [ ]:
thresholder = 300
names = []
for name, count in uniq_names.items():
    if count > thresholder:
        names.append([name,count])
names = sorted(names, key=lambda x: x[1], reverse=True)

In [ ]:
print('Уникальных имен функций: ', len(names))

### Фильтрация имен функций по значению

В выборку попадают функции, которые сложно классифицировать по их сниппету, поэтому такие имена функций исключаются из выборки.

In [ ]:
bad_names = ['main', 'run', 'init',
            'iter', 'enter','exit',
            'hash', 'del',
            'nonzero', 'func', 'foo', '_run']
names = list(filter(lambda name: name[0] not in bad_names and len(name[0]) > 2, names))

In [ ]:
print('Уникальных имен функций после чистки на неподходящие имена: ', len(names))

In [ ]:
names[:5]

In [ ]:
# Избавление от информации подсчета
names = [name for name, count in names]

### Фильтрация функций по выбранным именам

Все функции из файла `functions.json` будут отфильтрованы по имени и записаны в новый файл `filtered_functions.json`.

In [ ]:
pos = 0
functions, pos = load_portion_functions(FUNCTIONS_PATH, pos, 1)

print('Progress:')
while True:
    functions, pos = load_portion_functions(FUNCTIONS_PATH, pos, batch)
    if not functions:
        break
        
    append_functions_infile(
        FILTERED_FUNCTIONS_PATH, 
        filter_functions_name(functions, names)
    )
    print('.', end='')